<a href="https://colab.research.google.com/github/ZackAkil/Hands-on-Convolutional-Neural-Networks-Prediction-Interpretability/blob/master/Hands_on_Convolutional_Neural_Networks_%2B_Prediction_Interpretability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠🔍 Hands on Convolutional Neural Networks + Prediction Interpretability

This notebook will walk you through the process of building a CNN model for predicting [American Sign Language (**ASL**) Guestures](https://en.wikipedia.org/wiki/American_manual_alphabet) using [Keras](https://keras.io/), and then do prediction interpretability using [SHAP](https://github.com/slundberg/shap).

First we will import our data

---
**Practicle note ⚡**: becuase we are importing data direct from Cloud Storage into Colab, the data never touches our local computer and is leveraging the super fast  download speeds between Google services.

Download the data using the following command:

```bash
!gsutil cp "gs://fun-datasets/asl_mnist.csv" .
```

In [0]:
# [TASK] downlaod data using gsutil 



Now that the CSV file of our data in donwloaded, we can load it in using [pandas](https://pandas.pydata.org/):

```python
# this is how you import pandas
import pandas as pd

# then read in the CSV file 
data = pd.read_csv("FILE NAME OF CSV")

```

In [0]:
# [TASK] import pandas and read in csv 




### Looking at the data
You should have a look at your data:

 ```python
# display the top couple of rows of a pandas dataframe
my_data.head()
 ```

In [0]:
# [TASK] display the first couple of rows of your data using .head()




## Let's split out data into our inputs (images) and our outputs (ASL labels)

```python
# assign the label column to a varible y
y = data['label'].values

# assign the rest to a varible X
X = data.drop('label', axis = 1).values / 255.
```

In [0]:
# [TASK] split your data into X (inputs) and y (outputs)




## Let's actually look at one of these images
Print out the first image:
```python
print(X[0])
```
#### WHAT DO YOU EXPECT TO SEE?


In [0]:
# [TASK] print the first image (what do you expect)




## Where's our image!? 😡

So we tried to print our image but got a long list of numbers, WTF?

Well this is becuase it's easy to store pixel lists in CSV files. 

What we have todo is reshape the list into a matrix (image shape), then we can view it like a normal image. 

Todo that we need to know the original dimensions of the image.... it's **`28x28`** (you're welcome 😉)

There is a handy function in [numpy](https://www.numpy.org/) that will do this "reshaping" for us:


![alt text](https://storage.googleapis.com/random-assets/images/reshape.png)


So let's reshape the first image again, then display it using [matplotlib](https://matplotlib.org/):

```python
#import numpy
import numpy as np

# import matplotlib
import matplotlib.pyplot as plt

# reshape image
img = np.reshape(X[0], (original image height, original image width))

# display image
plt.imshow(img)
```




In [0]:
#[TASK] display the first image propery











OK assuming you've displayed one image correctly lets look at a couple of images:

Run the following code to see the first 5 images:

In [0]:
# display the first 5 images

# loop 5 times
for i in range(5):
  # reshape image
  img = np.reshape(X[i], (28, 28))
  # display image
  plt.imshow(img)
  plt.show()

Cool! We can see the images, but what do they mean??

Let's display them along side their labels:

In [0]:
# display the first 5 images with their labels

for i in range(5):

  # display label
  print(y[i])

  img = np.reshape(X[i], (28, 28))
  plt.imshow(img)
  plt.show()

Ok so are labels are **numbers**, not **letters**? 😕

I want to see something like this:

![alt text](https://storage.googleapis.com/kagglesdsdata/datasets/3258/5337/amer_sign2.png?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1561653295&Signature=hyygVqicdnvcUYesCK7%2FhMI12AtlnAS0BemhN4IWXEzLs4rLxTRU8Vre09S%2BozCb0YAbteiUXn70YVrr5S8tQGHf2oYYyyNkxBqNJYAXTPSUPVB0Ntr1YlTFUI467XqTGdyzxUZSm3qHYaGbwMr7FWLdUvsNuf7G9CmcHmp4%2BjjNFJHgH6Urawi19CVu%2BWBWmtJXKw4FB2nB%2FZvXZAqf8M2A%2BAur2k%2BBRCG7l4HCH63CUiKT8zHGE85RnjilDrcgzYuPE4KJbqmuKnkHch7FdXK3JraDAIqZldq92aiQsdgtDT7TciPzsCKfSxBBSMfqBhIFWihr0lNarz10en9sQA%3D%3D)

So we will have to convert our numbers to letters:

I've created a handy function to do just that below:

In [0]:
#  display first 5 images with the letters they represent:

# create function to convert number to letter
def number_to_letter(number):
  return chr(number+65)

for i in range(5):
  
  # display (letter) label
  print(number_to_letter(y[i]))

  img = np.reshape(X[i], (28, 28))
  plt.imshow(img)
  plt.show()

Use this chart to see if they are correct:

![alt text](https://storage.googleapis.com/kagglesdsdata/datasets/3258/5337/american_sign_language.PNG?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1561662820&Signature=NSMus25rd68OldMYcS2eiSR4Pe0rdycmn%2BL19ww5tM7YBeOPp6jS39Ktbqhae6IS7IeOroQ3iQziZF2LJAPXH7WDDhh2wU%2BkPDRibhTpbd9kdGaWn6IPqMpsyAcoNnSomK1mCQJ80VjcEcOlYrVT8V35ZhqtdY6YKFQpgZKXNnmEDGKaWt%2BpVe3sX6meIXTlOXo1csfBMzFTd1405wiYyYe97hEe9oDZDonHFSHjf5WpmRM7UXkjvRa9tb1mHOixZIf4NlJEYivNakh9F0LzLQs6dFegNZRpbqNsZj8qeutZYyrSI56HE4%2F%2FRBxMSXgMr5pf%2F%2FeyBIaS1eYhwcHbGw%3D%3D)

They are? yes? cool, we can proceed...

## Train/Test spliting our data
Like students being testing in a class room, if you give them the same test twise they'll just need to memorise the answers without actually learning the underlying consept. Thus when then they enter the real world and are faced with unseen problem, they will have no clue. 

So we will split our data into **training** data which the CNN will get to see and learning from, and **testing** data that it dosn't get to see during training, but we will use to make sure the CNN is actually learning.

We can use the [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) module from [sklearn](https://scikit-learn.org) to do this for us:

```python
# import module 
from sklearn.model_selection import train_test_split

# while we are here, let's reshape all of our images at once
X_reshaped = np.reshape(X, (len(X), original image height, original image width, 1))

# split our data
X_train, X_test, y_train, y_test = train_test_split(
    X_reshaped, y, test_size=0.20, random_state=42)
```

In [0]:
# [TASK] import the module, reshape all of the images, perform the train/test split







Let's have a peek at the dimensions (shape) of our data:

In [0]:
# let's quickly check the dimensions of our data
print("training input data - ", X_train.shape)
print("testing input data - ", X_test.shape)

print("training output data - ", y_train.shape)
print("testing output data - ", y_test.shape)

## Hold the phone! What does our output data look like?

Print out the first 10 outputs:

```python
print(y_train[:10])
```


In [0]:
# [TASK] print out the first 10 outputs





### Hmmmmmm, we have an issue with this:
<img height=300px src="https://storage.googleapis.com/random-assets/images/not_impressed_doggo.jpg">

Although we did agree that it's fine that our labels are numbers (becuase we can easily convert them to the ASL letters), our Convolutional Neural Network agreed to no such thing! In fact our CNN is alot more picky, it wants them as a [One-hot-encoded](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f) matrix!!!

It's all becuase of the structure of the CNN! Specifically the last layer, which is a singel node for each possible label (every letter of the alphabet in this case):

![alt text](https://storage.googleapis.com/random-assets/images/one_hot_cnn.png)

So let's quickly convert our number labels to the right format for the CNN (One-hot-encoded):

In [0]:
# convert our output numbers to the one-hot-encoded version
import keras

# get our data again
X_train, X_test, y_train, y_test = train_test_split(
    X_reshaped, y, test_size=0.20, random_state=42)

# encode the outputs (y's) as one-hot
y_train = keras.utils.to_categorical(y_train, 25)
y_test = keras.utils.to_categorical(y_test, 25)

Let's peek at teh new shape of our out data:

In [0]:
# let's quickly check the dimensions of our data
print("training input data - ", X_train.shape)
print("testing input data - ", X_test.shape)

print("training output data - ", y_train.shape)
print("testing output data - ", y_test.shape)

Ok but what does it actually look like now:

Print out the first 10 outputs:
```python
print(y_train[:10])
```

In [0]:
# [TASK] print out the first 10 outputs






### That might look ugly to us, but beautiful to our Convolutional Neural Network 😍

# Time to build the Convolutional Neural Network!!!

![alt text](https://storage.googleapis.com/random-assets/images/excited_pup.gif)

What's not that exciting is that we are just going to copy and past the same CNN that is in the [keras github for solving the mnist handwriting problem](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py). 

### This is pragmatic ML: find a model that does something similar and copy it!

*machine learning is all about adapting to new patterns*

here's teh code to build our model:


```python
# build new Convolutional Neural Network!

#import the nessasary keras modules
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

# start a new model
model = Sequential()

# create input layer (be sure to tell it the shape of data to expect)
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(ORIGINAL WIDTH OF IMAGE, ORIGINAL HEIGHT OF IMAGE, 1)))

# add soem more layers, why not!
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# you can change all of the previous stuff (except the input layer) and the following "output" layer
# be sure to fill in the shape of our output layer
model.add(Dense(THE WIDTH OF OUR ONE-HOT-ENCODED OUTPUT MATRIX, activation='softmax'))

# compile our model
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
```

In [0]:
# [TASK] build new Convolutional Neural Network!




















##<font color=red> Don't worry about any warnings above</font> 


## Let's train our CNN
Use the following code to train the CNN:

```python
# train our CNN on our training data for 5 epochs (cycles through our dataset)
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

# print out the final accuracy scores of our CNN          
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

```

In [0]:
# [TASK] run the training process











## Let's look at some predictions from our newly trained CNN:

In [0]:
# fetch predictions for the first 10 TEST images (images not seen by CNN)
model.predict_classes(X_test[:10])

In [0]:
# fetch predictions, convert them to letters
predictions = [number_to_letter(l) for l in model.predict_classes(X_test[:10])]
predictions

Let's actually look at the predictions alongside the input images and see if they are correct:

Use this chart to see if they are correct:

![alt text](https://storage.googleapis.com/kagglesdsdata/datasets/3258/5337/american_sign_language.PNG?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1561662820&Signature=NSMus25rd68OldMYcS2eiSR4Pe0rdycmn%2BL19ww5tM7YBeOPp6jS39Ktbqhae6IS7IeOroQ3iQziZF2LJAPXH7WDDhh2wU%2BkPDRibhTpbd9kdGaWn6IPqMpsyAcoNnSomK1mCQJ80VjcEcOlYrVT8V35ZhqtdY6YKFQpgZKXNnmEDGKaWt%2BpVe3sX6meIXTlOXo1csfBMzFTd1405wiYyYe97hEe9oDZDonHFSHjf5WpmRM7UXkjvRa9tb1mHOixZIf4NlJEYivNakh9F0LzLQs6dFegNZRpbqNsZj8qeutZYyrSI56HE4%2F%2FRBxMSXgMr5pf%2F%2FeyBIaS1eYhwcHbGw%3D%3D)

In [0]:
# show predictions with input images

for i in range(10):
  print("prediction = ", predictions[i])
  plt.imshow(X_test[i,:,:,0])
  plt.show()

# Congrats on building a sign language detecting Convolutional Neural Network ✋

If you found that useful: feel free to let use know with a tweet:

<a href="https://twitter.com/intent/tweet?text=Learning+about+interpreting+%23CNNs+using+the+SHAP+library+with+%23keras+on+%40GoogleColab+notebooks.+%0D%0AThanks+%40datalondon+%26+%40zackakil+for+the+awesome+meetup%21+%23datalondon+%23datascience+%23machinelearning+%0D%0Ahttps%3A%2F%2Frebrand.ly%2FCLDS-meetup-june25"><img src="https://storage.googleapis.com/events_public_polong/CLDS_Meetups/CLDS_tweet_20190625.png"></a>

# But wait! There's more! Let's see what our CNN sees! 🧐

There is a big push for more understanding around the kind of model that you have just built. [SHAP](https://github.com/slundberg/shap) is a tool that helps you do just that, by highlighting the areas of an image that the Convolutional Neural Network used most in making its' prediction.


![alt text](https://storage.googleapis.com/random-assets/images/shap.png)

## Let's run it on our CNN for the test data:

But first we need to install it in Colab (becuase it is not a standard ML package)

Install SHAP with teh following command:
```bash
!pip install shap 
```

In [0]:
# [TASK] install SHAP






## We'll cut to the chase becuase it's currently abit fiddely to use

straight from the [SHAP documentation](https://github.com/slundberg/shap):

---
*Predictions for two input images are explained in the plot [below]. Red pixels represent positive SHAP values that increase the probability of the class, while blue pixels represent negative SHAP values the reduce the probability of the class. By using ranked_outputs=2 we explain only the two most likely classes for each input (this spares us from explaining all [25] classes).*



In [0]:
import shap
import keras.backend as K

layer_to_explain = model.layers[1].input
data_to_explain = X_test[:10]

sess = K.get_session().run(layer_to_explain, {model.layers[0].input: X_train[:300]})
explainer = shap.GradientExplainer( (layer_to_explain, model.layers[-1].output), sess)

sess2 = K.get_session().run(layer_to_explain, {model.layers[0].input: data_to_explain })
shap_values, indexes = explainer.shap_values(sess2, ranked_outputs=2)

shap.image_plot(shap_values,  data_to_explain, np.vectorize(number_to_letter)(indexes))

# There you go!, You've built a CNN, and explained it's predictions!! 🏁